In [ ]:
%store -r
import os
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitoralert

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL',DKUBE_URL),token=os.getenv("DKUBE_USER_ACCESS_TOKEN",TOKEN))

### Import deployment

In [ ]:
api.import_deployment(name=MONITOR_NAME)

### Model Monitor

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME)

In [ ]:
mt=ModelType.Classification
dc_t=DatasetClass.Train

In [ ]:
text_file = open("transform_data.py", "r")
#read whole file to a string
script = text_file.read()
#close file
text_file.close()

In [ ]:
mm.update_modelmonitor(model_type=str(mt),data_timezone="utc")

#### TRAINING Details

In [ ]:
if DATA_SOURCE == "local" or DATA_SOURCE == "aws_s3":
    training_data = f'{DKUBE_BASE_DATASET}:'+ DKUBEUSERNAME
    train_data_version = 'v1:'+api.get_dataset_versions(
        DKUBEUSERNAME,
        DKUBE_BASE_DATASET)[0]['version']['uuid']
    prediction_data = MONITOR_NAME +'-predict:'+ DKUBEUSERNAME
    labelled_data = MONITOR_NAME +'-groundtruth:'+ DKUBEUSERNAME

if DATA_SOURCE == 'local':
    predict_data_version = 'v1:'+api.get_dataset_versions(
        DKUBEUSERNAME,
        MONITOR_NAME+'-predict')[0]['version']['uuid']
    labelled_data_version = 'v1:'+api.get_dataset_versions(
        DKUBEUSERNAME,
        MONITOR_NAME+'-groundtruth')[0]['version']['uuid']

if DATA_SOURCE == "aws_s3":
    if PREDICT_DATASET_FORMAT == "tabular":
        predict_data_format = str(DatasetFormat.Tabular)
    if PREDICT_DATASET_FORMAT == "cloudevents":
        predict_data_format = str(DatasetFormat.Cloudevents)
    if PREDICT_DATASET_FORMAT == "sagemaker":
        predict_data_format = str(DatasetFormat.Sagemaker)  

if DATA_SOURCE == "sql":
    training_data = f'{DKUBE_BASE_DATASET}:'+ DKUBEUSERNAME

### Add Training Dataset

In [ ]:
if DATA_SOURCE == 'sql':
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from titanic",transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from titanic_predict",date_suffix="yyyy/mm/dd/hh")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from titanic_gt",predict_col="Survived",groundtruth_col="GT_target",timestamp_col="timestamp")

if DATA_SOURCE == 'local':
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),version=train_data_version,transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,data_format=str(DatasetFormat.Tabular),version=predict_data_version,date_suffix="none")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,data_format=str(DatasetFormat.Tabular),version=labelled_data_version,predict_col="Survived",groundtruth_col="GT_target",timestamp_col="timestamp")

if DATA_SOURCE == 'aws_s3':
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),version=train_data_version,transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,data_format=predict_data_format,date_suffix="yyyy/mm/dd/hh")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,data_format=str(DatasetFormat.Tabular),predict_col="Survived",groundtruth_col="GT_target",timestamp_col="timestamp")

### Add Drift monitoring details

In [ ]:
mm.update_drift_monitoring_details(enabled=True,frequency=5,algorithm='auto')

### Create Model Monitor

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

#### Extracting ID of the model monitor

In [ ]:
id = api.modelmonitor_get_id(MONITOR_NAME)
id

### Update Schema

In [ ]:
api.modelmonitor_update_schema(id,label='Survived',schema_class='categorical',schema_type="prediction_output",selected=False)
api.modelmonitor_update_schema(id,label='PassengerId',schema_class='continuous',schema_type="row_id",selected=False)
api.modelmonitor_update_schema(id,label='timestamp',schema_class='continuous',schema_type="timestamp",selected=False)

api.modelmonitor_update_schema(id,label='Age',schema_class='continuous',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='Fare',schema_class='continuous',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='SibSp',schema_class='continuous',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='Parch',schema_class='continuous',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='Pclass',schema_class='categorical',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='Sex_male',schema_class='categorical',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='Sex_female',schema_class='categorical',schema_type='input_feature')


### Performance Monitoring

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME)
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=5,soft_thresholds={"accuracy":{"hard":0.890476152,"soft":0.816269806},"precision":{"hard":0.8795999999999999,"soft":0.8063},"f1_score": {"hard":0.67929912,"soft":0.62269086}})

In [ ]:
api.modelmonitor_update(id,mm)

### Add Alerts

#### Feature Alerts

In [ ]:
alert = DkubeModelmonitoralert(name='age_alert')
alert.update_alert(alert_class = 'feature_drift',feature='Age',threshold=0.02)
api.modelmonitor_add_alert(id,alert)

In [ ]:
alert = DkubeModelmonitoralert(name='fare_alert')
alert.update_alert(alert_class = 'feature_drift',feature='Fare',threshold=0.05)
api.modelmonitor_add_alert(id,alert)

#### Performance Alerts

In [ ]:
alert = DkubeModelmonitoralert(name='metrics_alert')
alert.update_alert(alert_class = 'performance_decay',metric = 'accuracy',threshold=0.70)
api.modelmonitor_add_alert(id,alert)

#### 6. Start the Model Monitor

In [ ]:
api.modelmonitor_start(id)

#### 7. Retraining

In [ ]:
%store -r
id = api.modelmonitor_get_id(MONITOR_NAME)

if INPUT_TRAIN_TYPE == 'retraining':
    api.modelmonitor_stop(id)
    
    training_data = f'{RETRAINING_DATASET}:'+DKUBEUSERNAME
    data_dict = api.get_dataset_versions(DKUBEUSERNAME,RETRAINING_DATASET)[0]['version']
    train_data_version = data_dict['name']+":"+data_dict['uuid']
    
    mm=DkubeModelmonitor(name=MONITOR_NAME)
    mm.update_datasources(name=training_data,data_class=str(DatasetClass.Train),version=train_data_version)
    api.modelmonitor_update(id,mm)
    
    
    ### Start the model monitor
    api.modelmonitor_start(id)



#### 8. Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(id)